In [230]:
# IMPORT LIBRARIES #
import pandas as pd
import pygsheets
import numpy as np

In [231]:
# FLAGS #

#Set to 1 to print diagnostic info
debugFlag = 0
#Set to 1 to pull info from RawNames sheet, join GPS, update, and export
updateGPSFlag = 0

In [232]:
# IMPORT DATA #

gc = pygsheets.authorize(service_file='file.json')
sh = gc.open('DashData')

if debugFlag:
    for x in sh:
        print(x)

for x in sh:
    title = x.title
    if(title=="DeliveriesOld"):
        import_sheets_OldDelDF = x.get_as_df()
    elif(title=="Deliveries"):
        import_sheets_DelDF = x.get_as_df()
    elif(title=="Days"):
        import_sheets_DaysDF = x.get_as_df()
    elif(title=="Weeks"):
        import_sheets_WeeksDF = x.get_as_df()
    elif(title=="RawNames"):
        import_sheets_RawNamesDF = x.get_as_df()
    elif(title=="GPS"):
        #for import
        import_sheets_GPSDF = x.get_as_df()
        #for export
        import_wks_gps = x
    else:
        print("Unused worksheet detected: ", x)

Unused worksheet detected:  <Worksheet 'Import' index:6>
Unused worksheet detected:  <Worksheet 'Lifetime Deliveries' index:7>
Unused worksheet detected:  <Worksheet 'Days Old' index:8>
Unused worksheet detected:  <Worksheet 'Weeks Old' index:9>


In [280]:
# Allows for disjoint cells, one for import, one for reset to import value (for ease of testing)

#DF from sheets
sheets_OldDelDF   = import_sheets_OldDelDF.copy()
sheets_DelDF      = import_sheets_DelDF.copy()
sheets_DaysDF     = import_sheets_DaysDF.copy()
sheets_WeeksDF    = import_sheets_WeeksDF.copy()
sheets_RawNamesDF = import_sheets_RawNamesDF.copy()
sheets_GPSDF      = import_sheets_GPSDF.copy()

#Sheets, ah right can't use copy on a worksheet object, might need an alternate
#wks_gps           = import_wks_gps.copy()

In [281]:
# FUNCTIONS #

# TODO: develop function to take avg distance and duration for stacked orders
def processStacks(df):
    return df

def dollars_to_floats(df, columns):
    for x in columns:
        df[x] = df[x].str.replace('$', '')
        df[x] = df[x].str.replace(',', '')
        df[x] = df[x].astype(float)
        
def hours_to_floats(df, columns):
    for x in columns:
        for index, row in df.iterrows():
            active = row[x]
            activeH = float(active[:active.index(":")])
            activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
            active = activeH + activeM
            df.at[index, x] = active
        df[x] = df[x].astype(float)
            
            

#Currently only valid for 2023
def date_to_datetime(df, column):
    for index, row in df.iterrows():
        df.at[index, column] = pd.Timestamp('2023/' + row[column])
    df[column] = pd.to_datetime(df[column])
    #print("TEST")
    #print(df.dtypes)
        
        
#Currently only valid for 2023
# Currently unused function
# Would we actually want this to have different dates? it's really the date the shift started
# So maybe, keep date same, but certain analysis might require that difference to be handled
def time_to_timestamp(df, DateCol, TimeCols):
    for index, row in df.iterrows():
        df.at[index, TimeCols[0]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[0]])
        df.at[index, TimeCols[1]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[1]])

In [282]:
# UPDATE GPS WORKSHEET #

if updateGPSFlag:
    testingDF = sheets_RawNamesDF.copy()
    testingDF = testingDF.drop_duplicates(subset=["Restaurant Name", "RAW DATA"])
    testingDF = testingDF[["Restaurant Name", "RAW DATA", "Note"]]
    newNamesDF = testingDF.merge(sheets_GPSDF, how='left')
    for index, row in newNamesDF.iterrows():
        if isinstance(row['Address'], float):
            if not row['Note'] == "":
                newNamesDF.at[index, "Address"] = row["Note"]
            else:
                print("Note not found for:", index)
        if (row['Address'] == ""):
            newNamesDF.at[index, "Resolved"] = "0"
        else:
            newNamesDF.at[index, "Resolved"] = "1"
    newNamesDF = newNamesDF.drop(columns=["Note"])
    import_wks_gps.set_dataframe(newNamesDF,(1,1))

In [283]:
# START PROCESSING #

In [284]:
# COMBINE DELIVERY DATA #
sheets_frames = [sheets_OldDelDF, sheets_DelDF]
sheets_result = pd.concat(sheets_frames)
sheets_result.reset_index(drop=True, inplace=True)

In [285]:
# FUNCTIONS AND OTHER SHORT PROCESSING STEPS #


#All
dollars_to_floats(sheets_result, ["Total", "Base", "Tip", "Peak Bonus"])
#V3 only
dollars_to_floats(sheets_DelDF, ["Total", "Base", "Tip", "Peak Bonus"])
#Days
dollars_to_floats(sheets_DaysDF, ["Total", "Base", "Tip"])
hours_to_floats(sheets_DaysDF, ["Dash", "Active"])
date_to_datetime(sheets_DaysDF, "Date")
sheets_DaysDF["Day of Week"] = sheets_DaysDF["Date"].dt.day_name()
#Weeks
dollars_to_floats(sheets_WeeksDF, ["Total", "Base", "Tip", "Adj pay", "Other", "Pre Total"])
hours_to_floats(sheets_WeeksDF, ["Dash", "Active"])
# pretotalactiveratio refers to weekly pre adjustment pay / time spent on deliveries
sheets_WeeksDF["pretotalactiveratio"] = round(sheets_WeeksDF["Pre Total"]/sheets_WeeksDF["Active"], 2)

/tmp/ipykernel_4404/588475045.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')
/tmp/ipykernel_4404/588475045.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


In [286]:
#date_to_timestamp(sheets_DaysDF, "Date")

In [287]:
# BEGIN ADJ PAY SECTION #

# This establishes the estimated pay adjustment estimates by delivery, date, and week
# This is calculated using time spent delivering and distance approximations
# The real adjusted pay is based on weekly values
# I suspect the primary cause for difference between estimate and real values stems from 
#how I estimate mileage vs how the app tracks mileage

# DFs are copied to avoid excluding dropped values in later analysis



#if debugFlag:
#    sheets_DelDF.tail()

In [288]:
#Per delivery estimations
#sheets_DelDF1 = sheets_DelDF.copy()

#Convert to float if possible, else convert to NaN
#sheets_DelDF1["Duration"] = pd.to_numeric(sheets_DelDF1["Duration"], errors='coerce')
sheets_DelDF["Duration"] = pd.to_numeric(sheets_DelDF["Duration"])
#Drop NaN rows
#sheets_DelDF1 = sheets_DelDF1[sheets_DelDF1["Duration"].notna()]
#Calculate estimate adjustment per delivery
sheets_DelDF["Est Adj"] = (sheets_DelDF["Duration"]/60*18
                           + sheets_DelDF["Distance"]*(0.34) ) - sheets_DelDF["Base"]
#Duration is over 60 because its value is in minutes, values from day and week are in hours

#if debugFlag:
#    sheets_DelDF.tail()

In [289]:
#if debugFlag:
#    sheets_DaysDF.tail()

In [290]:
#Per day estimations
#sheets_DaysDF1 = sheets_DaysDF.copy()
sheets_DaysDF = sheets_DaysDF[sheets_DaysDF["Version"] == 3]
sheets_DaysDF["Est Adj"] = (sheets_DaysDF["Active"]*(18) 
                            + sheets_DaysDF["Active Mileage"]*(0.34) ) - sheets_DaysDF["Base"]

In [291]:
#if debugFlag:
#    sheets_DaysDFv3.tail()

In [292]:
#Per week estimations
#sheets_WeeksDF1 = sheets_WeeksDF.copy()
sheets_WeeksDF = sheets_WeeksDF[sheets_WeeksDF["Version"] == 3] #totals doesn't have v3 so its dropped
sheets_WeeksDF["Est Adj"] = (sheets_WeeksDF["Active"]*(18) 
                            + sheets_WeeksDF["Active Mileage"]*(0.34) ) - sheets_WeeksDF["Base"]

In [293]:
#if debugFlag:
#    sheets_WeeksDFv3.tail()

# Sample values

#
if debugFlag:
    print("Del DF Sample\n")
    print(sheets_DelDF.iloc[-1])
    print("\nDays DF Sample\n")
    print(sheets_DaysDF.iloc[-1])
    print("\nWeeks DF Sample\n")
    print(sheets_WeeksDF.iloc[-1])
    
    
# END ADJ PAY SECTION #

In [301]:
print(list(sheets_DelDF.columns.values))
print()
print(list(sheets_DaysDF.columns.values))
print()
print(list(sheets_WeeksDF.columns.values))
print()
print(sheets_DaysDF.dtypes)

['ID', 'Date', 'Restaurant Name', 'Total', 'Base', 'Tip', 'Peak Bonus', 'Stacked', 'Start Time', 'Distance', 'End Time', 'Duration', 'Version', 'Est Adj']

['Date', 'Total', 'Base', 'Tip', 'Start (24)', 'End (24)', 'Active', 'Dash', 'Offers', 'Deliveries', 'Mileage Start', 'Mileage End', 'Total Mileage', 'Active Mileage', 'Version', 'Pauses', 'WeekID', 'Day of Week', 'Est Adj']

['Dates', 'Base', 'Tip', 'Adj pay', 'Other', 'Pre Total', 'Total', 'Active', 'Dash', 'Deliveries', 'Total Mileage', 'Active Mileage', 'Version', 'WeekID', 'pretotalactiveratio', 'Est Adj']

Date              datetime64[ns]
Total                    float64
Base                     float64
Tip                      float64
Start (24)                object
End (24)                  object
Active                   float64
Dash                     float64
Offers                     int64
Deliveries                 int64
Mileage Start             object
Mileage End               object
Total Mileage             object

In [295]:
# END PROCESSING #
# START INITIAL EXPLORATION #

In [296]:
#Percentage of deliveries taken that tip
numUntipped = (sheets_result["Tip"] == 0.0).sum()
numTipped = (sheets_result["Tip"] != 0.0).sum()
percTipped = round((numTipped/(numUntipped+numTipped)), 4)
print(percTipped*100, "% of accepted deliveries included tip")

#Average tip, including and excluding untipped deliveries
TipAvg = round(sheets_result["Tip"].mean(), 2)
print("$", TipAvg, "average tip value including orders with no tip")
TipAvg2 = round(sheets_result[sheets_result["Tip"] !=0]["Tip"].mean(), 2)
print("$", TipAvg2, "average tip value excluding orders with no tip")

# Overall hourly rate
hourlyRate = round(sheets_WeeksDF.iloc[-1]["Total"]/sheets_WeeksDF.iloc[-1]["Dash"], 2)
print("Overall hourly pay rate is $", hourlyRate)

# Max and Average values by Delivery, Day, and Week
# Some of these values are wrong, because I have not yet implemented a stack value handler
print()
print("Daily Max Values")
print(sheets_DaysDF[["Total", "Base", "Tip", "Active", "Dash", "Deliveries"]].max())
print(sheets_DaysDF[sheets_DaysDF["Version"] == 3][["Total Mileage", "Active Mileage"]].max(), "\n" )
print("Daily Average Values")
print(sheets_DaysDF[["Total", "Base", "Tip", "Active", "Dash", "Deliveries"]].mean())
print(sheets_DaysDF[sheets_DaysDF["Version"] == 3][["Total Mileage", "Active Mileage"]].mean(), "\n" )
print()
# Some of these values are wrong, because I have not yet implemented a stack value handler
print("Weekly Max Values")
#
print(sheets_WeeksDF[["Base", "Tip", "Adj pay", "Pre Total", "Total", "Active", "Dash", "Deliveries"]].max())
#
print(sheets_WeeksDF[sheets_WeeksDF["Version"] == 3][["Total Mileage", "Active Mileage"]].max(), "\n")
print("Weekly Average Values")
#
print(sheets_WeeksDF[["Base", "Tip", "Adj pay", "Pre Total", "Total", "Active", "Dash", "Deliveries"]].mean())
#
print(sheets_WeeksDF[sheets_WeeksDF["Version"] == 3][["Total Mileage", "Active Mileage"]].mean(), "\n")
print()
# Some of these values are wrong, because I have not yet implemented a stack value handler
print("Delivery Max Values")
#
print(sheets_DelDF[["Distance", "Duration"]].max())
print(sheets_result[["Total"]].max(), "\n")
print("Delivery Average Values")
#
print(sheets_DelDF[["Distance", "Duration"]].mean())
print(sheets_result[["Total"]].mean(), "\n")

# END INITIAL EXPLORATION #

90.74 % of accepted deliveries included tip
$ 4.12 average tip value including orders with no tip
$ 4.54 average tip value excluding orders with no tip
Overall hourly pay rate is $ 25.54

Daily Max Values
Total         333.57
Base          191.50
Tip           149.53
Active          9.67
Dash           13.60
Deliveries     33.00
dtype: float64
Total Mileage       151
Active Mileage    229.2
dtype: object 

Daily Average Values
Total         163.776393
Base           91.659836
Tip            72.116557
Active          5.198525
Dash            7.483115
Deliveries     17.508197
dtype: float64
Total Mileage     97.071429
Active Mileage    93.614286
dtype: float64 


Weekly Max Values
Base           637.25
Tip            475.09
Adj pay        240.02
Pre Total     1112.34
Total         1358.61
Active          38.60
Dash            51.23
Deliveries     124.00
dtype: float64
Total Mileage       633
Active Mileage    619.4
dtype: object 

Weekly Average Values
Base          288.305556
Tip       

In [297]:
# Restaurants rankable by avg or mean base/tip/total
# Yes, total is almost always base+tip, but total INCLUDES peak pay, which may be desirable information

tempDF = sheets_result.copy()
#The line below drops restaurants that only appear once, we may no longer want this to happen
#Commented out for now, may make sense to use when considering outliers for averages
#tempDF = tempDF[sheets_result["Restaurant Name"].duplicated(keep=False) == True]
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Base": [np.mean, np.sum], 
                                                  "Tip": [np.mean, np.sum],
                                                  "Total": [np.mean, np.sum],
                                                 'Restaurant Name': 'size'})
tempDF.columns = ['Avg Base', 'Sum Base', 'Avg Tip', 
                  'Sum Tip', 'Avg Total', 'Sum Total', "Deliveries"]

tempDF = tempDF.round({'Avg Base': 2, 
                       'Sum Base': 2,
                       'Avg Tip': 2,
                       'Sum Tip': 2,
                       'Avg Total': 2,
                       'Sum Total': 2})

#Change sort value to see differences
tempDF1 = tempDF.index.copy()
tempDF.sort_values("Sum Total", ascending=False)






#RESTAURANTS DF

,Avg Base,Sum Base,Avg Tip,Sum Tip,Avg Total,Sum Total,Deliveries
Restaurant Name,,,,,,,
Chick-fil-A,5.70,689.75,3.61,436.39,9.31,1126.14,121
Taco Bell,4.79,315.87,3.44,226.75,8.70,574.12,66
McDonald's,4.59,316.50,3.30,227.42,7.90,544.92,69
Dave's Hot Chicken,5.24,256.75,4.36,213.50,9.60,470.25,49
Chipotle,4.69,248.50,3.52,186.42,8.24,436.92,53
...,...,...,...,...,...,...,...
Everbowl,4.00,4.00,2.00,2.00,6.00,6.00,1
Kung Fu Noodle,5.50,5.50,0.00,0.00,5.50,5.50,1
Papa Johns,4.00,4.00,1.00,1.00,5.00,5.00,1


In [298]:
#Dollars Earned vs Miles Driven is another factor I wanted to consider in the overall earnings equation

# Grab v3 deliveries, drop rows without known distance values
# Group by restaurant name, sum on total and distance
# Evaluate total pay over total distance as new column
# Sort low to high


#This actually uses sheets_DelDF, it could use DelDF1, but also they could also be sheets_DelDF
tempDF = sheets_DelDF.copy()
#tempDF = tempDF[tempDF.Distance!="-"]
tempDF["Distance"] = tempDF["Distance"].astype(float)
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Total": "sum", "Distance": "sum"})
tempDF = tempDF.round({'Base': 2, 'Tip': 2})
tempDF["DollarMilesRatio"] = tempDF["Total"]/tempDF["Distance"]
#tempDF.sort_values(by="DollarMilesRatio")
tempDF2 = tempDF.index.copy()
tempDF


#JOIN WITH RESTAURANTS DF


#160 in total, 115 in v3? so 45 restaurants unique to v1 and 2??

#Can I select results where version !=3, get rest count name no
#can we select rest from temp1 not in temp2?

,Total,Distance,DollarMilesRatio
Restaurant Name,,,
7-Eleven,122.37,92.4,1.324351
Advanced Auto Parts,13.75,4.6,2.989130
Albertsons,34.17,8.8,3.882955
Angry Chickz,11.25,7.9,1.424051
Anita's Mexican Restaurant & Cantina,28.25,16.9,1.671598
...,...,...,...
Wendy's,45.25,17.8,2.542135
Wings-N-Things,14.75,7.9,1.867089
Wingstop,27.50,17.9,1.536313


In [299]:
tempDF3 = tempDF1[~tempDF1.isin(tempDF2)].dropna(how = 'all')
tempDF3 #These are the restaurants from just v1, v2
#Thought there were too many but, there were actually a lot that haven't been seen since v3

Index(['Acai Superfruits', 'Beach Hut Deli', 'Bloom Haus',
       'Brooklyn Boyz Pizza', 'Chili's Grill & Bar', 'Chin's Szechwan',
       'Cream Of The Crop', 'Epic Wings', 'Erika's Mexican Food & Seafood',
       'Everbowl', 'FTD', 'Firehouse Subs', 'Greek Corner Cafe',
       'Guanatos City Mexican Restaurant', 'Handel's Homemade Ice Cream',
       'Harney Sushi', 'Jamba', 'Joe's Crab Shack', 'Krispy Kreme',
       'Mikko Sushi', 'North Park Produce Bakery and Grill',
       'Ocean Thai Cuisine', 'PETCO', 'Pit Stop Diner', 'Pollos Maria',
       'Quik Wok', 'Sizzler', 'Sonic Drive-In', 'Super Mega Dilla',
       'TZONE Oceanside', 'Tapioca Express', 'TenderFix by Noah Schnapp',
       'The Cheesecake Factory', 'The Cheesesteak Grill', 'The Meltdown',
       'The Poke Cafe', 'Tony's Fresh Mexican Food',
       'Umi Japanese Grill & Sushi', 'Village Kabob', 'Wienerschnitzel',
       'Wild Burger'],
      dtype='object', name='Restaurant Name')

In [300]:
# Average $/hour and tip amount by day of week
# NOTE: This excludes pay adjustments

tempDF = sheets_DaysDF.copy()

# Can move this up to formatting function calls
# Also, check if necessary to convert from string to timestamp to datetime
#date_to_timestamp(tempDF, "Date")
#tempDF["Date"] = pd.to_datetime(tempDF["Date"])

#Adds column to DaysDF for the day of the week (Monday, Thursday, etc.)
#tempDF["Day of Week"] = tempDF["Date"].dt.day_name()

tempDF = tempDF.groupby(["Day of Week"]).agg(
    {"Dash": "sum", 
     "Total": [np.mean, np.sum], 
     "Tip": "sum", 
     "Deliveries": "sum"})
tempDF.columns = ['Sum Dash', 'Avg Daily Total', 'Sum Total', 'Sum Tip', 'Sum Deliveries']

tempDF["$/hr"] = tempDF["Sum Total"]/tempDF["Sum Dash"]
tempDF["Avg Delivery Tip"] = tempDF["Sum Tip"]/tempDF["Sum Deliveries"]
tempDF["Avg Delivery Total"] = tempDF["Sum Total"]/tempDF["Sum Deliveries"]

tempDF = tempDF.round({'Sum Dash': 2, 
                       'Avg Total': 2, 
                       'Sum Total':2, 
                       'Sum Tip': 2,
                       '$/hr': 2,
                       'AvgTip': 2,
                       'AvgTotal':2})

display(tempDF)

,Sum Dash,Avg Daily Total,Sum Total,Sum Tip,Sum Deliveries,$/hr,Avg Delivery Tip,Avg Delivery Total
Day of Week,,,,,,,,
Friday,135.57,189.533125,3032.53,1385.78,316,22.37,4.385380,9.596614
Monday,17.85,125.823333,377.47,158.97,41,21.15,3.877317,9.206585
Saturday,112.83,185.460714,2596.45,1144.95,270,23.01,4.240556,9.616481
Sunday,49.58,147.347500,1178.78,545.53,122,23.78,4.471557,9.662131
Thursday,72.83,140.773636,1548.51,656.01,176,21.26,3.727330,8.798352
Wednesday,67.81,139.624444,1256.62,507.87,143,18.53,3.551538,8.787552
